In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\feder\AppData\Local\Temp\ipykernel_1588\3184247913.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [9]:
def find_tournaments_by_id(id):
    time.sleep(1)
    if id==-1:
        return pd.DataFrame(columns=["game", "reg_players", "max_players", "date"])
    driver.get('https://boardgamearena.com/tournamentlist?d&time=0&prestige=0&type=0&players_per_match_min=0&players_per_match_max=0&gamecateg=3&status=future&game='+str(id)+'&tournament_i_registered=0&full=false')
    html= driver.page_source
    soup = BeautifulSoup(html)
    tournament_list=soup.find(id="tournament_list")
    games_list_html=tournament_list.find_all(class_="tournaments-list-result__gamename")
    registered_players_html=tournament_list.find_all(class_="tournaments-list-result__players-reg")
    max_players_html=tournament_list.find_all(class_="tournaments-list-result__players-max")
    date_tournament_html = tournament_list.find_all(class_="tournaments-list-result__date")

    games_list = []
    reg_players = []
    max_players = []
    date_tournament = []
    for i in range(len(games_list_html)):
        games_list.append(games_list_html[i].get_text())
        reg_players.append(registered_players_html[i].get_text())
        max_players.append(max_players_html[i].get_text())
        date_tournament.append(date_tournament_html[i].get_text().split(' ')[1])

    zipped = list(zip(games_list, reg_players, max_players, date_tournament))
    df = pd.DataFrame(zipped, columns=["game", "reg_players", "max_players", "date"])
    return df

In [10]:
def find_id_from_gamename(gamename):
    driver.get('https://boardgamearena.com/gamepanel?game='+gamename)
    time.sleep(1)
    html= driver.page_source
    soup = BeautifulSoup(html)
    x=soup.find_all(class_="bgabutton bgabutton_blue bga-button-inner flex-1 truncate svelte-1154gir")
    if not x:
        return -1
    id = str(x[1]).split("=")[3].split('"')[0]
    return id

In [11]:
df_games = pd.read_json("game.json")
df_tournaments = pd.DataFrame(columns=["game", "reg_players", "max_players", "date"])
for name in df_games["name"]:
    time.sleep(1)
    df = find_tournaments_by_id(find_id_from_gamename(name.lower().replace(" ", "")))
    pd.concat([df_tournaments, df], axis=1)

df_tournaments

IndexError: list index out of range